# Implementación de una Red Neuronal

En el modelo de la red neuronal se tiene comúnmente una función:
        $$o(Wx+b)$$
donde $W$ es un vector de pesos, $b$ es el bias y $o:\mathbb{R}^n \to \{0,1\}$ una función de salida que determina la clase del elemento al que pertenece el vector $x$. 

En este caso, se tomarán $m$ capas ocultas de tal forma que se tenga una función de pre-activación que definiremos como: $$a_i(x) = W_i\cdot h_{i-1}(x)+b_i, i=1,...,m$$

donde cada función de activación en las capas ocultas esté dada por $$h_i(x) = g(a_i(x))$$
Ydonde $g \equiv S$ donde la función $S: \mathbb{R}^n \to (0,1)$ es una función sigmoide definida como:
$$S(x) = \frac{1}{a+\exp(-x)}$$ 
Aunque debe resaltarse que la función de activación puede ser de otro tipo, como la tangente hiperbólica. De está forma, la función de salida será: $$h_m(x) = o(a_m(x))$$

In [61]:
import numpy as np

#Definimos la función sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Para obtener la activación en la capa de salida, utilizaremos la función Softmax:
$$f_c(x) = p(y=y_c|x) = \frac{\exp(W h_m(x)+b)}{\sum_{x'}\exp(W h_m(x')+b)}$$ 

Obtendremos la derivada de la función $-log(f(x)_c)$ como: $$\frac{\delta(-log(f(x)_c))}{\delta a_i(x)_c} = f(x)_c - \delta_{yc}$$

Se utilizará el algoritomo de Stochastic Gradient Descent (SGD) para actualizar los parámetros:
$$ W_i \leftarrow W_i - \eta \cdot \nabla(-log(f(x)_c) + \gamma \cdot r(W_i))$$

In [62]:
def build_model(X, y, nn_hdim, its=10000, print_loss=False, reg_l=0.01, a=0.1):
    np.random.seed(0)
    #El número de rasgos que representan cada vector
    nn_input_dim = X.shape[1]
    #El total de clases que arrojará
    output_dim = len(set(y))
    #El número de ejmplos
    num_examples = len(X)

    #Con estos datos se generan los vectores de parámetros
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim,  output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, output_dim ))
    
    model = {}
    for i in xrange(0, its):
        #Defino la función Wx+b y la función de activación g=tanh
        z1 = np.dot(X,W1) + b1
        a1 = sigmoid(z1)
        z2 = np.dot(a1,W2) + b2

        #Uso la función Softmax para determinar las probabilidades de las clases
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        #Cada clase correspondiente a los datos se le resta 1
        probs[range(num_examples),y] -= 1

        #Se obtienen las derivadas para los parámetros en las capas ocultas
        dW2 = a1.T.dot(probs)
        db2 = np.sum(probs,axis=0, keepdims=True)
        
        #Se obtienen las derivadas para los parámetros de entrada
        d2 = probs.dot(W2.T) * (1-np.power(a1,2))
        dW1 = np.dot(X.T,d2)
        db1 = np.sum(d2, axis=0)
        
        #Se suma un valor de regularización a los parametros
        dW2 += reg_l*W2
        dW1 += reg_l*W1

        #Se utiliza el método de estochastic Gradient Descent
        W1 += -a*dW1
        b1 += -a*db1
        W2 += -a*dW2
        b2 += -a*db2

        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}

        if print_loss and i % 100 == 0:
            print "Perdida en la iteracion %i: %f" %(i, loss(model))

    return model

Una vez obtenido el modelo podemos obtener la función de predicción en la capa de salida de la red neuronal como:

$$\hat{y} = \arg\max_c p(y=y_c|x)$$

In [63]:
def predict(model, X):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    z1 = np.dot(X,W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1,W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)

Ahora podemos tomar algunos ejemplos para entrenar la red neuronal.

In [64]:
X = np.array([[0,1],[1,0],[0,0],[1,1]])
y = np.array([0,0,1,1])

model = build_model(X,y,2,print_loss=False)

print model['W1']
print model['b1']
print model['W2']

[[  7.65946476  -7.67624769]
 [ 10.22302928  10.24525071]]
[[-6.52446281  1.12684182]]
[[ 2.04374153 -2.04371309]
 [-2.04014573  2.04017124]]


Y finalmente, podemos evaluar un conjunto de puntos.

In [65]:
x = np.array([0,0])
print predict(model, x)

[1]
